In [15]:
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer

data = pd.read_csv('2024-2025 Syllabi.csv')
data.head()

,Semester,Department,Course Number,Title,Instructor,Syllabus,Course Code
0,"2024, Fall",ARC,696,Advanced Design.,"Lickwar, Phoebe",graduate program landscape architectureschool ...,ARC 696
1,"2024, Fall",ARC,696,Advanced Architectural Design.,"Garrison, Michael",fall arc r advanced design studiomichael garri...,ARC 696
2,"2024, Fall",ARC,696,Advanced Architectural Design.,"Garrison, Michael",fall arc r advanced design studiomichael garri...,ARC 696
3,"2024, Fall",ARC,696,Advanced Architectural Design.,"Kyriakou, Kyriakos",polykatoikia refilling domino kyriakos kyriako...,NaN
4,ARC 696,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.dropna(inplace=True)
data.head()

,Semester,Department,Course Number,Title,Instructor,Syllabus,Course Code
0,"2024, Fall",ARC,696,Advanced Design.,"Lickwar, Phoebe",graduate program landscape architectureschool ...,ARC 696
1,"2024, Fall",ARC,696,Advanced Architectural Design.,"Garrison, Michael",fall arc r advanced design studiomichael garri...,ARC 696
2,"2024, Fall",ARC,696,Advanced Architectural Design.,"Garrison, Michael",fall arc r advanced design studiomichael garri...,ARC 696
11,"2024, Fall",ARC,695S,Advanced Architectural Design: Integrative Stu...,"Haettasch, Martin",form form performance entertainment city stage...,ARC 695S
12,"2024, Fall",ARC,695S,Advanced Architectural Design: Integrative Stu...,"Fajkus, Matt",form form performance entertainment city stage...,ARC 695S


In [ ]:
# Model Creation
model = SentenceTransformer('all-MiniLM-L6-v2')
syll_emmbeddings = model.encode(data['Syllabus'].tolist(), show_progress_bar=True)

In [12]:
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['Syllabus'])

user_input = input("What topic are you interested in? ")
clean_input = ' '.join(simple_preprocess(user_input.lower()))
input_vector = tfidf.transform([clean_input])

data['Relevance'] = cosine_similarity(tfidf_matrix, input_vector).flatten()
recommended = data.sort_values(by='Relevance', ascending=False).head(10)
print('It sounds like you might be interested in the following courses:')
row_count = 0
title = ''
for index, row in recommended.iterrows():
    if row['Title'] != title:
        print(f"- {row['Course Code']}: {row['Title']} (Relevance Score: {row['Relevance']:.4f})")
        title = row['Title']
        row_count += 1
        if row_count > 5:
            break
    else:
        continue

It sounds like you might be interested in the following courses:
- CRW 315D: Playwriting I. (Relevance Score: 0.3015)
- ART 325M: Intermediate Print: Lithography. (Relevance Score: 0.1954)
- ART 325G: Intermediate Print: Serigraphy. (Relevance Score: 0.1640)
- ART 310P: Introduction to Print. (Relevance Score: 0.1605)
- M 361K: Introduction to Real Analysis. (Relevance Score: 0.1509)
- KIN 355: Media and Public Relations in Sport. (Relevance Score: 0.1502)


In [ ]:
user_input = input("What topic are you interested in? ")
clean_input = ' '.join(simple_preprocess(user_input.lower()))

input_embedding = model.encode([clean_input])

data['Relevance'] = cosine_similarity(syll_emmbeddings, input_embedding).flatten()
recommended = data.sort_values(by='Relevance', ascending=False).head(10)

print('It sounds like you might be interested in the following courses:')
row_count = 0
title = ''
for index, row in recommended.iterrows():
    if row['Title'] != title:
        print(f"- {row['Course Code']}: {row['Title']} (Relevance Score: {row['Relevance']:.4f})")
        title = row['Title']
        row_count += 1
        if row_count > 5:
            break
    else:
        continue

Batches: 100%|██████████| 399/399 [08:48<00:00,  1.33s/it]


It sounds like you might be interested in the following courses:
- POL 325: Third-Year Polish I. (Relevance Score: 0.5117)
- GER 612: Accelerated Second-Year German: Readings in Modern German. (Relevance Score: 0.4950)
- CMS 355K: Intercultural Communication. (Relevance Score: 0.4950)
- GER 506: First-Year German I. (Relevance Score: 0.4927)
- GER 612: Accelerated Second-Year German: Readings in Modern German. (Relevance Score: 0.4915)
- GER 507: First-Year German II. (Relevance Score: 0.4872)


In [20]:
with open('syllabus_model.pkl', 'wb') as f:
    pickle.dump((model, syll_emmbeddings, data), f)

In [10]:
import pickle
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity

with open('syllabus_model.pkl', 'rb') as f:
    model, syll_emmbeddings, data = pickle.load(f)

user_input = input("What topic are you interested in? ")
clean_input = ' '.join(simple_preprocess(user_input.lower()))
input_embedding = model.encode([clean_input])

data['Relevance'] = cosine_similarity(syll_emmbeddings, input_embedding).flatten()
recommended = data.sort_values(by='Relevance', ascending=False).head(10)

print('It sounds like you might be interested in the following courses:')
row_count = 0
title = ''
for index, row in recommended.iterrows():
    if row['Title'] != title:
        print(f"- {row['Course Code']}: {row['Title']} (Relevance Score: {row['Relevance']:.4f})")
        title = row['Title']
        row_count += 1
        if row_count > 5:
            break
    else:
        continue

It sounds like you might be interested in the following courses:
- GOV 337F: Politics, Economy, and Society of Contemporary Brazil. (Relevance Score: 0.4709)
- GOV 314E:  ics of Social and Political Thought. (Relevance Score: 0.4520)
- PPE 320: Foundations of Philosophy, Politics, and Economics. (Relevance Score: 0.4284)
- I B 372T: Global Political Economy. (Relevance Score: 0.4284)
- ECO 339K: International Trade and Investment. (Relevance Score: 0.4229)
- PPE 301: Introduction to Philosophy, Politics, and Economics. (Relevance Score: 0.4172)
